In [1]:
import pandas as pd

# Potentially problematic models
Models that are known to have failed/have long download times - download everything else first before attempting these


In [2]:
problem_models = ["CESM2","CESM2-WACCM","KACE-1-0-G","TaiESM1","KIOST-ESM"]

# Download lists of available data from synda

```
fpath=/rds/general/user/cb2714/home/01_wwa/10_misc/00_list-models

synda search project=CMIP6 frequency=day experiment_id=historical,ssp585 variable=tas,tasmax,tasmin,pr -l 5000 > $fpath/vars-cmip6.txt
synda search project=CMIP6 frequency=day experiment_id=highresSST-present,highresSST-future variable=tas,tasmax,tasmin,pr -l 5000 > $fpath/vars-highresmip.txt

synda search project=CMIP6 frequency=day experiment_id=historical,ssp585 variable=tas,pr,sfcWind,hurs -l 10000 > $fpath/fwi-cmip6.txt
synda search project=CMIP6 frequency=day experiment_id=highresSST-present,highresSST-future variable=tas,pr,sfcWind,hurs -l 10000 > $fpath/fwi-highresmip.txt
```

## Available runs

In [3]:
vset = "fwi"
ds = "highresmip"

df = pd.read_table(vset+"-"+ds+".txt", sep = "\.", engine = "python", header = None,
                   names = [0,"activity","institute","source_id","experiment","variant_label","table", "variable", "grid", "version"])

In [4]:
mdl_xvar = df[["source_id", "variant_label", "variable", "experiment"]].drop_duplicates()

# filter only models with both time periods
mdl_xvar = mdl_xvar.groupby(["source_id", "variant_label", "variable"]).count().reset_index()
mdl_xvar = mdl_xvar.loc[mdl_xvar.experiment > 1]

In [5]:
# list models with all available data for run r1i1p1f1 (this is the single most commonly used label)
df_r1 = mdl_xvar.loc[mdl_xvar.variant_label == "r1i1p1f1"]
ct_r1 = pd.crosstab(index = df_r1.variable, columns = [df_r1.source_id, df_r1.variant_label]).transpose()
ct_r1

,variable,hurs,pr,sfcWind,tasmax
source_id,variant_label,,,,
CMCC-CM2-HR4,r1i1p1f1,0,1,0,0
CMCC-CM2-VHR4,r1i1p1f1,0,1,0,0
EC-Earth3P,r1i1p1f1,0,1,0,1
EC-Earth3P-HR,r1i1p1f1,1,1,1,1
FGOALS-f3-H,r1i1p1f1,0,1,1,0
FGOALS-f3-L,r1i1p1f1,0,1,1,1
GFDL-CM4C192,r1i1p1f1,1,1,1,1
HadGEM3-GC31-HM,r1i1p1f1,1,1,1,1
HadGEM3-GC31-LM,r1i1p1f1,1,1,1,1


In [6]:
# list models with all necessary variables
mlist = [x[0] for x in ct_r1.loc[ct_r1.sum(1) == ct_r1.shape[1]].index]
",".join(mlist)

'EC-Earth3P-HR,GFDL-CM4C192,HadGEM3-GC31-HM,HadGEM3-GC31-LM,HadGEM3-GC31-MM,HiRAM-SIT-HR,HiRAM-SIT-LR,MPI-ESM1-2-XR,MRI-AGCM3-2-H,MRI-AGCM3-2-S,NICAM16-7S,NICAM16-8S,NICAM16-9S'

In [7]:
# list models that don't have an r1i1p1f1 variant - which variants could we add to the list?
df_rx = mdl_xvar.loc[~mdl_xvar.source_id.isin(df_r1.source_id)]
ct_rx = pd.crosstab(index = df_rx.variable, columns = [df_rx.source_id, df_rx.variant_label]).transpose()
ct_rx.loc[ct_rx.sum(1) == ct_rx.shape[1]]

variable                     hurs  pr  sfcWind  tasmax
source_id     variant_label                           
CNRM-CM6-1    r10i1p1f2         1   1        1       1
              r2i1p1f2          1   1        1       1
              r3i1p1f2          1   1        1       1
              r4i1p1f2          1   1        1       1
              r5i1p1f2          1   1        1       1
              r6i1p1f2          1   1        1       1
              r7i1p1f2          1   1        1       1
              r8i1p1f2          1   1        1       1
CNRM-CM6-1-HR r10i1p1f2         1   1        1       1
              r2i1p1f2          1   1        1       1
              r3i1p1f2          1   1        1       1
              r4i1p1f2          1   1        1       1
              r5i1p1f2          1   1        1       1
              r6i1p1f2          1   1        1       1
              r7i1p1f2          1   1        1       1
              r8i1p1f2          1   1        1       1
              r9i1p1f2          1   1        1       1

In [31]:
# ensemble sizes
pd.crosstab(index = mdl_xvar.source_id, columns = mdl_xvar.variable)

variable,hurs,pr,sfcWind,tasmax
source_id,,,,
CMCC-CM2-HR4,0,1,0,0
CMCC-CM2-VHR4,0,1,0,0
CNRM-CM6-1,10,10,8,10
CNRM-CM6-1-HR,9,10,9,10
EC-Earth3P,0,1,0,1
EC-Earth3P-HR,1,1,1,1
FGOALS-f3-H,0,1,1,0
FGOALS-f3-L,0,1,1,1
GFDL-CM4C192,1,1,1,1


In [32]:
# expand models & ensemble members with variables
pd.crosstab(index = mdl_xvar.variable, columns = [mdl_xvar.source_id, mdl_xvar.variant_label]).transpose()

variable                       hurs  pr  sfcWind  tasmax
source_id       variant_label                           
CMCC-CM2-HR4    r1i1p1f1          0   1        0       0
CMCC-CM2-VHR4   r1i1p1f1          0   1        0       0
CNRM-CM6-1      r10i1p1f2         1   1        1       1
                r1i1p1f2          1   1        0       1
                r2i1p1f2          1   1        1       1
                r3i1p1f2          1   1        1       1
                r4i1p1f2          1   1        1       1
                r5i1p1f2          1   1        1       1
                r6i1p1f2          1   1        1       1
                r7i1p1f2          1   1        1       1
                r8i1p1f2          1   1        1       1
                r9i1p1f2          1   1        0       1
CNRM-CM6-1-HR   r10i1p1f2         1   1        1       1
                r1i1p1f2          0   1        0       1
                r2i1p1f2          1   1        1       1
                r3i1p1f2          1   1        1       1
                r4i1p1f2          1   1        1       1
                r5i1p1f2          1   1        1       1
                r6i1p1f2          1   1        1       1
                r7i1p1f2          1   1        1       1
                r8i1p1f2          1   1        1       1
                r9i1p1f2          1   1        1       1
EC-Earth3P      r1i1p1f1          0   1        0       1
EC-Earth3P-HR   r1i1p1f1          1   1        1       1
FGOALS-f3-H     r1i1p1f1          0   1        1       0
FGOALS-f3-L     r1i1p1f1          0   1        1       1
GFDL-CM4C192    r1i1p1f1          1   1        1       1
HadGEM3-GC31-HM r1i1p1f1          1   1        1       1
HadGEM3-GC31-LM r1i1p1f1          1   1        1       1
HadGEM3-GC31-MM r1i1p1f1          1   1        1       1
HiRAM-SIT-HR    r1i1p1f1          1   1        1       1
HiRAM-SIT-LR    r1i1p1f1          1   1        1       1
MPI-ESM1-2-HR   r1i1p1f1          0   1        1       1
MPI-ESM1-2-XR   r1i1p1f1          1   1        1       1
MRI-AGCM3-2-H   r1i1p1f1          1   1        1       1
MRI-AGCM3-2-S   r1i1p1f1          1   1        1       1
NICAM16-7S      r1i1p1f1          1   1        1       1
NICAM16-8S      r1i1p1f1          1   1        1       1
NICAM16-9S      r1i1p1f1          1   1        1       1

## CORDEX


In [2]:
vset = "fwi"
ds = "cordex"

df = pd.read_table(vset+"-"+ds+".txt", sep = "\.", engine = "python", header = None,
                   names = [0,1,"domain","institute","driving_model","experiment","ensemble","rcm_name", "rcm_version", "frequency", "variable","version"])

In [3]:
mdl_xvar = df[["domain", "driving_model", "ensemble","rcm_name","rcm_version","variable", "experiment"]].drop_duplicates()

# # filter only models with both time periods
mdl_xvar = mdl_xvar.groupby(["domain", "driving_model", "ensemble","rcm_name","rcm_version","variable"]).count().reset_index()
mdl_xvar = mdl_xvar.loc[mdl_xvar.experiment > 1]

In [4]:
# list models with all available data
ct_r1 = pd.crosstab(index = mdl_xvar.variable, columns = [mdl_xvar.domain, mdl_xvar.driving_model, mdl_xvar.ensemble, mdl_xvar.rcm_name, mdl_xvar.rcm_version]).transpose()
ct_r1.loc[ct_r1.sum(1) == ct_r1.shape[1]]

variable                                                   hurs  pr  sfcWind  \
domain driving_model        ensemble rcm_name rcm_version                      
NAM-22 CCCma-CanESM2        r1i1p1   CRCM5    v1              1   1        1   
                                     CanRCM4  r2              1   1        1   
       MOHC-HadGEM2-ES      r1i1p1   REMO2015 v1              1   1        1   
                                     RegCM4   v4-4-rc8        1   1        1   
                                     WRF      v3-5-1          1   1        1   
       MPI-M-MPI-ESM-LR     r1i1p1   CRCM5    v1              1   1        1   
                                     REMO2015 v1              1   1        1   
                                     RegCM4   v4-4-rc8        1   1        1   
                                     WRF      v3-5-1          1   1        1   
       MPI-M-MPI-ESM-MR     r1i1p1   CRCM5    v1              1   1        1   
       NCC-NorESM1-M        r1i1p1   REMO2015 v1              1   1        1   
       NOAA-GFDL-GFDL-ESM2M r1i1p1   RegCM4   v4-4-rc8        1   1        1   
                                     WRF      v3-5-1          1   1        1   
NAM-44 CCCma-CanESM2        r1i1p1   CRCM5    v1              1   1        1   
                                     CanRCM4  r2              1   1        1   
                                     RCA4     v1              1   1        1   
       ICHEC-EC-EARTH       r12i1p1  RCA4     v1              1   1        1   
                            r3i1p1   HIRHAM5  v1              1   1        1   
       MOHC-HadGEM2-ES      r1i1p1   RegCM4   v4-4-rc8        1   1        1   
                                     WRF      v3-5-1          1   1        1   
       MPI-M-MPI-ESM-LR     r1i1p1   CRCM5    v1              1   1        1   
                                     RegCM4   v4-4-rc8        1   1        1   
                                     WRF      v3-5-1          1   1        1   
       MPI-M-MPI-ESM-MR     r1i1p1   CRCM5    v1              1   1        1   
       NOAA-GFDL-GFDL-ESM2M r1i1p1   RegCM4   v4-4-rc8        1   1        1   
                                     WRF      v3-5-1          1   1        1   

variable                                                   tasmax  
domain driving_model        ensemble rcm_name rcm_version          
NAM-22 CCCma-CanESM2        r1i1p1   CRCM5    v1                1  
                                     CanRCM4  r2                1  
       MOHC-HadGEM2-ES      r1i1p1   REMO2015 v1                1  
                                     RegCM4   v4-4-rc8          1  
                                     WRF      v3-5-1            1  
       MPI-M-MPI-ESM-LR     r1i1p1   CRCM5    v1                1  
                                     REMO2015 v1                1  
                                     RegCM4   v4-4-rc8          1  
                                     WRF      v3-5-1            1  
       MPI-M-MPI-ESM-MR     r1i1p1   CRCM5    v1                1  
       NCC-NorESM1-M        r1i1p1   REMO2015 v1                1  
       NOAA-GFDL-GFDL-ESM2M r1i1p1   RegCM4   v4-4-rc8          1  
                                     WRF      v3-5-1            1  
NAM-44 CCCma-CanESM2        r1i1p1   CRCM5    v1                1  
                                     CanRCM4  r2                1  
                                     RCA4     v1                1  
       ICHEC-EC-EARTH       r12i1p1  RCA4     v1                1  
                            r3i1p1   HIRHAM5  v1                1  
       MOHC-HadGEM2-ES      r1i1p1   RegCM4   v4-4-rc8          1  
                                     WRF      v3-5-1            1  
       MPI-M-MPI-ESM-LR     r1i1p1   CRCM5    v1                1  
                                     RegCM4   v4-4-rc8          1  
                                     WRF      v3-5-1            1  
       MPI-M-MPI-ESM-MR     r1i1p1   CRCM5    v1           

In [24]:
ct_r1.loc[ct_r1.sum(1) != ct_r1.shape[1]]

variable                                                    hurs  pr  sfcWind  \
domain driving_model         ensemble rcm_name rcm_version                      
NAM-22 CNRM-CERFACS-CNRM-CM5 r1i1p1   CRCM5    v1              0   1        0   
       NOAA-GFDL-GFDL-ESM2M  r1i1p1   CRCM5    v1              0   1        0   

variable                                                    tasmax  
domain driving_model         ensemble rcm_name rcm_version          
NAM-22 CNRM-CERFACS-CNRM-CM5 r1i1p1   CRCM5    v1                1  
       NOAA-GFDL-GFDL-ESM2M  r1i1p1   CRCM5    v1                1

# Monthly vs daily availability for potential intensity variables

```
synda search project=CMIP6 frequency=day experiment_id=historical,ssp585 variable=tos,ta,hus,psl -l 10000 > /rds/general/user/cb2714/home/01_wwa/10_misc/00_list-models/pi-cmip6.txt
synda search project=CMIP6 frequency=mon experiment_id=historical,ssp585 variable=tos,ta,hus,psl -l 10000 > /rds/general/user/cb2714/home/01_wwa/10_misc/00_list-models/pi-cmip6-monthly.txt

```


In [21]:
df_d = pd.read_table("pi-cmip6-daily.txt", sep = "\.", engine = "python", header = None,
                   names = [0,"activity","institute","source_id","experiment","variant_label","table", "variable", "grid", "version"])
df_d["frequency"] = "day"

df_m = pd.read_table("pi-cmip6-monthly.txt", sep = "\.", engine = "python", header = None,
                   names = [0,"activity","institute","source_id","experiment","variant_label","table", "variable", "grid", "version"])
df_m["frequency"] = "mon"

df = pd.concat([df_d, df_m])

In [22]:
mdl_xvar = df[["source_id", "variant_label", "variable", "experiment", "frequency"]].drop_duplicates()

In [26]:
# filter only models with both time periods
mdl_xvar = mdl_xvar.groupby(["source_id", "variant_label", "frequency", "variable"]).count().reset_index()
mdl_xvar = mdl_xvar.loc[mdl_xvar.experiment > 1]

In [30]:
# and also with all four variables
mdl_xvar = mdl_xvar.groupby(["source_id", "variant_label", "frequency"]).count().reset_index()
mdl_xvar = mdl_xvar.loc[mdl_xvar.experiment == 4]

In [33]:
ct1 = pd.crosstab(index = mdl_xvar.frequency, columns = [mdl_xvar.source_id, mdl_xvar.variant_label]).transpose()

In [37]:
# very small number of models where daily data is available but not monthly - no need to archive from daily
ct1.loc[(ct1.day == 1) & (ct1.mon == 0)]

frequency                   day  mon
source_id    variant_label          
CNRM-CM6-1   r3i1p1f2         1    0
EC-Earth3    r13i1p1f1        1    0
IPSL-CM6A-LR r14i1p1f1        1    0
             r2i1p1f1         1    0
             r33i1p1f1        1    0
             r3i1p1f1         1    0
             r4i1p1f1         1    0
             r6i1p1f1         1    0

In [54]:
# list of models with all variables available at monthly resolution for variant r1i1p1f1
df_mon = pd.merge(df_m, ct1.loc[(ct1.mon == 1)].reset_index(), left_on = ["source_id", "variant_label"], right_on = ["source_id", "variant_label"])
mlist_r1 = sorted(list(set(df_mon.loc[df_mon.variant_label == "r1i1p1f1", "source_id"])))
",".join(mlist_r1)

'AWI-CM-1-1-MR,BCC-CSM2-MR,CAMS-CSM1-0,CIESM,CMCC-CM2-SR5,CMCC-ESM2,CanESM5,EC-Earth3,EC-Earth3-CC,EC-Earth3-Veg,EC-Earth3-Veg-LR,FGOALS-f3-L,FGOALS-g3,FIO-ESM-2-0,IITM-ESM,INM-CM4-8,INM-CM5-0,IPSL-CM6A-LR,MIROC6,MPI-ESM1-2-HR,MPI-ESM1-2-LR,MRI-ESM2-0,NESM3,NorESM2-LM,NorESM2-MM,TaiESM1'

In [76]:
mlist_rx = [m for m in sorted(list(set(df_mon.loc[df_mon.variant_label != "r1i1p1f1", "source_id"]))) if not m in mlist_r1]
mlist_rx

['CESM2',
 'CNRM-CM6-1',
 'CNRM-CM6-1-HR',
 'CNRM-ESM2-1',
 'CanESM5-CanOE',
 'HadGEM3-GC31-LL',
 'HadGEM3-GC31-MM',
 'MCM-UA-1-0',
 'UKESM1-0-LL']